In [1]:
import os
import os.path
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kypher

input_path = None
output_path = '/data/table-understanding/tmp'
project_name = 'wikidata_fact_verification'

additional_files = {}

ck = ConfigureKGTK([])
ck.configure_kgtk(input_graph_path=input_path, 
                  output_path=output_path, 
                  project_name=project_name,
                  additional_files=additional_files)

User home: /nas/home/akthakur
Current dir: /data/table-understanding/wikidata-fact-verification
KGTK dir: /data/table-understanding
Use-cases dir: /data/table-understanding/use-cases


## Get intersection of wikidata and dbpedia facts

In [ ]:
kypher("""
    -i ../claims.wikibase-item.tsv.gz --as wiki
    -i ../wikidata_infobox.tsv --as dbp
    -i /data/table-understanding/labels.en.tsv.gz --as labels
    --match 'wiki:(p1)-[:P26]->(p2),
            dbp:(p1)-[:`property:spouse`]->(p3),
            labels:(p1)-[]->(n1_label),
            labels:(p2)-[]->(n2_label)'
    --where 'p2 = p3'
    --return 'distinct p1 as person, kgtk_lqstring_text(n1_label) as name, p2 as spouse, kgtk_lqstring_text(n2_label) as spouse_name'
    -o fact_verification_data.tsv
""")

## Get wikidata spouse data

In [4]:
%%time
kypher("""
    -i ../claims.wikibase-item.tsv.gz --as wiki
    -i /data/table-understanding/labels.en.tsv.gz --as labels
    --match 'wiki:(p1)-[:P26]->(p2),
            labels:(p1)-[]->(n1_label),
            labels:(p2)-[]->(n2_label)'
    --return 'distinct p1 as person, kgtk_lqstring_text(n1_label) as name, p2 as spouse, kgtk_lqstring_text(n2_label) as spouse_name'
    -o wiki_spouse_data.tsv
""")

CPU times: user 17.1 ms, sys: 15.2 ms, total: 32.3 ms
Wall time: 21.6 s


## Get dbpedia spouse data

In [5]:
%%time
kypher("""
    -i ../wikidata_infobox.tsv --as dbp
    -i /data/table-understanding/labels.en.tsv.gz --as labels
    --match 'dbp:(p1)-[:`property:spouse`]->(p2),
            labels:(p1)-[]->(n1_label),
            labels:(p2)-[]->(n2_label)'
    --return 'distinct p1 as person, kgtk_lqstring_text(n1_label) as name, p2 as spouse, kgtk_lqstring_text(n2_label) as spouse_name'
    -o dbpedia_spouse_data.tsv
""")

## Present in db pedia not in Wikidata

In [42]:
with open('./wiki_spouse_data.tsv', 'r') as f_wiki, open('./dbpedia_spouse_data.tsv', 'r') as f_dbp:
    dbp_lines = f_dbp.readlines()
    wiki_lines = f_wiki.readlines()
    
    only_dbp_spouse_data = [dbp_lines[0]] + list(set(dbp_lines) - set(wiki_lines))

len(only_dbp_spouse_data)

7316

In [44]:
with open('only_dbp_spouse_data.tsv', 'w') as f:
    for line in only_dbp_spouse_data:
        f.write(line)